In [ ]:
import pandas as pd
import numpy as np
import ruptures as rpt
import plotly.express as px
import ipywidgets as widgets
import matplotlib.pyplot as plt

pd.options.plotting.backend = "plotly"

%matplotlib inline

In [ ]:
%store -r df

In [ ]:
value = ''
fdf = df[(df.ageRange.notnull()) & (df.ageRange == value)] if value else df
fdf

In [ ]:
work_ages = ['25-34', '35-44','45-54','55-64']
df['work_age'] = df.ageRange.isin(work_ages)
#non_work_ages = ['18-24', '65-74','75-84','85-94']
#non_working_df = df[df.ageRange.isin(non_work_ages)]

In [ ]:
# fdf.pivot(index='hour', columns='series', values='steps')

(df.groupby(['hour','series'])
   .median().unstack('series')['steps'].plot.line(markers=True, line_shape='spline', height=200))
# grouped.pivot(index='hour', columns='series')

# grouped.plot(kind="line")
#fig = px.line(grouped, y="steps")
#fig.show()

In [ ]:
before = df[df['series'] == 'before']
plot = before.groupby(['hour','work_age']).median().unstack('work_age')['steps'].plot.line(title='Before working from home', line_shape='spline', height=200)
plot.update_layout(xaxis = dict(
        tickmode = 'array',
        tickvals = [0,24, 48, 72, 96, 120, 144, 168],
        ticktext = ['Mon', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun']
    ))

In [ ]:
after = df[df['series'] == 'after']
plot = (after.groupby(['hour','work_age'])
   .median().unstack('work_age')['steps'].plot.line(title='After working from home',line_shape='spline', height=200))
plot.update_layout(xaxis = dict(
        tickmode = 'array',
        tickvals = [0,24, 48, 72, 96, 120, 144, 168],
        ticktext = ['Mon', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun']
    ))

In [ ]:
series_df = fdf[fdf['series'] == 'before']

(series_df.groupby(['hour'])
   .median()['steps'].plot.line())

In [ ]:
series_df = fdf[fdf['series'] == 'after']

(series_df.groupby(['hour'])
   .median()['steps'].plot.line())

In [ ]:
steps_before = np.array(fdf[fdf['series'] == 'before'].groupby(['hour']).median()['steps'])
steps_after = np.array(fdf[fdf['series'] == 'after'].groupby(['hour']).median()['steps'])
# signal = np.stack((steps_before, steps_after), axis=1)
signal = steps_before

signal

In [ ]:
#n_samples, dim, sigma = 1000, 3, 4
#n_bkps = 4  # number of breakpoints
#signal, bkps = rpt.pw_constant(n_samples, dim, n_bkps, noise_std=sigma)
#bkps

In [ ]:
# detection
#cost = rpt.costs.CostL2()
model = "l2"
algo = rpt.Binseg(model=model).fit(signal)
my_bkps = algo.predict(n_bkps=3)

# display
rpt.display(signal, my_bkps, figsize=(17, 6))
plt.show()

df2 = fdf[fdf['series'] == 'before']
bkps_timestamps = df2.iloc[[0] + my_bkps[:-1] +[-1]].index
durations = (bkps_timestamps[1:] - bkps_timestamps[:-1])
d = durations.seconds/60/60
d_f = pd.DataFrame(d)


In [ ]:
new_signal = np.array([0,1,2,3,4,6,6,6,6,6,10,10,10,13,14,10,11,10,10,10,10,10,10,23])

cost = rpt.costs.CostL2()
model = "l2"
algo = rpt.Binseg(model=model).fit(new_signal)
my_bkps = algo.predict(n_bkps=3)

print(signal)
print(my_bkps)

# display
rpt.display(new_signal, my_bkps, figsize=(17, 6))
plt.show()